In [1]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import logging
import torch
import math
import time
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict

from transformers import BertTokenizer,BertModel,BertTokenizerFast
from utils.MIND import MIND

logger = logging.getLogger(__name__)

In [2]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'

# config.signal_length = 10
# config.scale = 'large'
# config.impr_size = 100
# config.mode = 'test'

path = config.path + 'MIND/MINDdemo_train/'
a = MIND(config, path + 'news.tsv', path + 'behaviors.tsv')

[2021-08-31 12:50:10,648] INFO (utils.MIND) encoding user behaviors of ../../../Data/MIND/MINDdemo_train/behaviors.tsv...
[2021-08-31 12:50:10,789] INFO (utils.MIND) encoding news of ../../../Data/MIND/MINDdemo_train/news.tsv...
[2021-08-31 12:51:27,106] INFO (utils.MIND) reducing news of ../../../Data/MIND/MINDdemo_train/news.tsv...
[2021-08-31 12:51:27,110] INFO (utils.utils) unmasking at least k...
[2021-08-31 12:51:27,420] INFO (utils.utils) deduplicating...


In [5]:
a.encoded_news[3][:110]

array([  101,  1996,  3465,  1997,  8398,  1005,  1055,  4681, 13184,
        1999,  1996, 19874,  1997,  5924,  1005,  1055,  2162,  8318,
        1012,  7332,  9587, 29358,  7231,  3215, 18652,  2058,  1037,
       27372,  1997,  5472,  8641,  2012,  1996,  2392,  2240,  1997,
        1996,  2162,  1999,  5924,  1012,  2279,  2000,  2032,  2001,
        2019,  4064, 10412, 16863,  2039,  2000,  7577, 17515,  2015,
        1010,  2525,  2566, 29278,  4383,  2007,  3674,  8198,  1012,
        2739,  2739, 11108,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0])

In [7]:
t.convert_ids_to_tokens(a.encoded_news[3][:110])

['[CLS]',
 'the',
 'cost',
 'of',
 'trump',
 "'",
 's',
 'aid',
 'freeze',
 'in',
 'the',
 'trenches',
 'of',
 'ukraine',
 "'",
 's',
 'war',
 'lt',
 '.',
 'ivan',
 'mo',
 '##lch',
 '##ane',
 '##ts',
 'peeked',
 'over',
 'a',
 'parapet',
 'of',
 'sand',
 'bags',
 'at',
 'the',
 'front',
 'line',
 'of',
 'the',
 'war',
 'in',
 'ukraine',
 '.',
 'next',
 'to',
 'him',
 'was',
 'an',
 'empty',
 'helmet',
 'propped',
 'up',
 'to',
 'trick',
 'sniper',
 '##s',
 ',',
 'already',
 'per',
 '##for',
 '##ated',
 'with',
 'multiple',
 'holes',
 '.',
 'news',
 'news',
 '##world',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PA

In [4]:
a.subwords[3][:110]

array([[ 0,  0],
       [ 1,  1],
       [ 2,  2],
       [ 3,  3],
       [ 4,  4],
       [ 5,  5],
       [ 6,  6],
       [ 7,  7],
       [ 8,  8],
       [ 9,  9],
       [10, 10],
       [11, 11],
       [12, 12],
       [13, 13],
       [14, 14],
       [15, 15],
       [16, 16],
       [17, 17],
       [18, 18],
       [19, 19],
       [20, 20],
       [20, 21],
       [20, 22],
       [20, 23],
       [21, 24],
       [22, 25],
       [23, 26],
       [24, 27],
       [25, 28],
       [26, 29],
       [27, 30],
       [28, 31],
       [29, 32],
       [30, 33],
       [31, 34],
       [32, 35],
       [33, 36],
       [34, 37],
       [35, 38],
       [36, 39],
       [37, 40],
       [38, 41],
       [39, 42],
       [40, 43],
       [41, 44],
       [42, 45],
       [43, 46],
       [44, 47],
       [45, 48],
       [46, 49],
       [47, 50],
       [48, 51],
       [49, 52],
       [49, 53],
       [50, 54],
       [51, 55],
       [52, 56],
       [52, 57],
       [52, 58

In [3]:
loader1 = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, sampler=Partition_Sampler(a,2,0))
records1 = list(loader1

In [7]:
records1[0]['his_attn_mask']

tensor([[[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [3]:
dic = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/bert/MINDdemo_train/news.pkl', 'rb'))

In [4]:
sb = dic['subwords_first']
sb

array([list([]),
       list([[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 6], [7, 7], [8, 8], [9, 9], [10, 10], [11, 11], [12, 12], [13, 13], [14, 14], [15, 15], [16, 16], [0, 0], [17, 18], [18, 19], [19, 20], [20, 21], [21, 22], [22, 23], [23, 24], [24, 25], [25, 26], [26, 27], [27, 28], [28, 29], [29, 30], [30, 31], [31, 32], [32, 33], [0, 0], [0, 0]]),
       list([[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 6], [7, 7], [8, 8], [9, 9], [10, 10], [11, 11], [12, 12], [13, 13], [14, 14], [15, 15], [16, 16], [17, 17], [18, 18], [19, 19], [0, 0], [20, 21], [21, 22], [22, 23], [23, 24], [24, 25], [25, 26], [26, 27], [27, 28], [28, 29], [0, 0], [0, 0]]),
       ...,
       list([[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 6], [7, 7], [8, 8], [9, 9], [10, 10], [11, 11], [12, 12], [13, 13], [14, 14], [15, 15], [16, 16], [17, 17], [18, 18], [19, 19], [0, 0], [20, 21], [21, 22], [22, 23], [23, 24], [24, 25], [25, 26], [26, 27], [27, 28], [28, 29], [29, 30], [30, 31], [3

In [17]:
np.unique(a[:,0]),a

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([[0, 0],
        [1, 1],
        [2, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [5, 6],
        [6, 7],
        [7, 8],
        [8, 9]]))

In [6]:
t = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [4]:
t.tokenize("I don't give a fuck")

['i', 'don', "'", 't', 'give', 'a', 'fuck']